In [1]:
import pandas as pd

meeting_note_csv_path = "../audio/싼기타_비싼기타_final.csv"
df_rttm = pd.read_csv(meeting_note_csv_path, sep='|')

display(df_rttm)

FileNotFoundError: [Errno 2] No such file or directory: '../audio/싼기타_비싼기타_final.csv'

In [ ]:
# 이름 넣기
name_dict = {
    "SPEAKER_00": "AI",
    "SPEAKER_01": "이성용",
}

df_rttm["name"] = df_rttm["speaker_id"].apply(lambda x: name_dict[x])

display(df_rttm)

In [ ]:
meeting_note_txt = df_rttm[['start', 'end', 'name', 'text']].to_json(orient='records', force_ascii=False)
print(meeting_note_txt)

In [ ]:
system_prompt = f'''
너는 회의 내용을 요약하는 봇이다. 아래 회의록을 읽고, 주요 내용을 요약하라.
결과는 마크다운 형식으로 작성한다.

아래 형식에 맞추어 작성하라.

# 회의 제목

## 주요 내용

## 참석자별 입장

## 결정 사항   

=============== 이하 회의록 ===============

{ meeting_note_txt }
'''

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
# OpenAI API를 사용해 요약 결과 생성
response = client.chat.completions.create(
    model='gpt-4o',
    temperature=0.1,
    messages=[
        {"role": "system", "content": system_prompt},
    ]
)

# (4) 요약 결과를 파일로 저장
summary = response.choices[0].message.content 
summary = summary.strip() # 좌우 공백 제거

print(summary)

In [ ]:
with open('../audio/guitar_summary.md', 'w', encoding='utf-8') as f:
    f.write(summary)

In [ ]:
df_meeting_note = df_rttm[['start', 'end', 'name', 'text']].copy()
df_meeting_note.dropna(inplace=True)

meeting_note_dict = df_meeting_note.to_dict(orient='records')  # 딕셔너리 형태로 변환
meeting_note_dict

In [ ]:
for row in meeting_note_dict:    
    if not isinstance(row['text'], str): # 텍스트가 아닌 다른 형식인 경우 통과
        continue

    correction_system_prompt = f'''
        너는 주어진 회의 녹취록을 수정 및 보완하는 봇이다.
        주어진 회의 녹취록은 STT로 작성된 결과이므로, 이 중에 오류가 있는 부분을 찾아내고 수정하라.

        표현, 말투는 최대한 원본과 일치하도록 유지하되, 잘못 받아적은 내용을 수정하라.
        원본의 내용을 최대한 살리되, 잘못된 내용만 수정하라. 
        어떤 경우에는 A라는 사람이 말한 내용을 B가 말한 것으로 잘못 기록된 경우도 있을 수 있다. 이런 경우에는 문맥을 고려하여 수정하라.

        원본 내용 중 빠지거나, 추가되거나, 잘못 기록된 부분을 찾아내어 수정하라. 수정된 메시지 이외에는 아무것도 작성하지 말라.
        --------------------------------
        회의 요약문 : {summary}
        --------------------------------
        회의 녹취록 전문: {df_meeting_note.to_json(orient='records', force_ascii=False)}
        --------------------------------
        확인할 메시지 원본: {row['text']}
    '''

    response = client.chat.completions.create(
        model='gpt-4o',
        temperature=0.1,
        messages=[
            {"role": "system", "content": correction_system_prompt},
        ]
    )

    correction = response.choices[0].message.content
    if ':' in correction:  # “수정: “ 과 같은 형식으로 반환되는 경우 해결 
        correction = correction.split(':')[-1]
        
    correction = correction.strip() # 좌우 공백 제거

    print(row['name'])
    print(row['text'])
    print(correction)
    print('-----------------')

    row['corrected_text'] = correction

In [ ]:
# meeting_note_dict json 파일로 저장
import json

with open('../audio/guitar_meeting_note_corrected.json', 'w', encoding='utf-8') as f:
    json.dump(meeting_note_dict, f, ensure_ascii=False, indent=4)

In [ ]:
md_template = ""

for row in meeting_note_dict:
    md_template += f"- **{row['name']}** : {row['corrected_text']}\n"

with open('../audio/guitar_meeting_note_corrected.md', 'w', encoding='utf-8') as f:
    f.write(md_template)

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document()   # (1)  docx 객체 생성
document.add_heading('회의록', level=1) # (2) 제목 추가

table = document.add_table(rows=1, cols=2) # (3) 테이블 생성
table.style = 'Table Grid' # (4) 테이블 스타일 지정

hdr_cells = table.rows[0].cells # (5) 첫 번째 행에 셀 추가
hdr_cells[0].text = 'Speaker' # (6) 첫 번째 셀에 Speaker 추가
hdr_cells[1].text = 'Content'   # (7) 두 번째 셀에 Content 추가

# (8) 수정된 녹취록 파일을 읽어들여서 테이블에 추가
for row in meeting_note_dict:
    row_cells = table.add_row().cells
    row_cells[0].text = row['name']
    row_cells[1].text = row['corrected_text']


document.save('../audio/guitar_meeting_note_corrected.docx') # (9) docx 파일 저장